# Notebook E-tivity 3 CE4021 Task 2

Student name: Collin Czernel

Student ID: 

<hr style=\"border:2px solid gray\"> </hr>

## Imports

In [ ]:
#None

If you believe required imports are missing, please contact your moderator.

<hr style=\"border:2px solid gray\"> </hr>

## Task 2

Use the below information to create a Naive Bayes SPAM filter. Test your filter using the messages in new_emails. You may add as many cells as you require to complete the task.

Create a Naive Bayes Classifier to filter incoming mail for SPAM. Write code using Bayes' Rule to determine whether the messages contained in new_emails are HAM or SPAM. Compare the decisions your classifier takes with the label associated with the messages. If time permits, add the code required to allow your classifier to learn from the email messages contained in new_emails. Note that this functionality is required to be graded in the Exemplary category.

In [1]:
previous_spam = ['send us your password', 'review our website', 'send your password', 'send us your account']
previous_ham = ['Your activity report','benefits physical activity', 'the importance vows']
new_emails = {'spam':['renew your password', 'renew your vows'], 'ham':['benefits of our account', 'the importance of physical activity']}

So initial thoughts - split up the emails and look for key words. Learning: function at end that if the email was marked "HAM", add key phrases to "previous ham" list.

From core resources on Bayes Theorem, $P(H|E) = \frac{P(H) \cdot P(E|H)}{P(E)}$

Breaking this down:
$P(H)$ is the probability a hypothesis is true (before any evidence), $P(E|H)$ is the probability of seeing the evidence that the hypothesis is true,
$P(E)$ is the probability of seeing the evidence, $P(H|E)$ is the probability a hypothesis is true given some evidence.

For every word that is in the previous_spam list, we assign a probability such that if that word appears again, it counts (see assignment for how I did this). 

In [9]:
saved_spam = [] # for the "learning" part
saved_ham = [] # for the "learning" part

def split_list(email_list): # Basically, for each new email in the list splitting it up into individual words 
    words = [] # Storage for our new words 
    for i in email_list:
        words.extend(i.split())
    return words

spam_words = split_list(previous_spam) # getting all the words in previous spam emails
ham_words = split_list(previous_ham) # getting all the words in previous ham emails

def count_words(word_list): # Counting the words in the original spam, ham lists. This is dynamic so should allow for updating. 
    word_count = {} # Setting up dictionary to save them as like 'word': 5, meaning 'word' appeared 5 times
    for word in word_list: # Word is starting to look like not an actual word... running loop for through word_list
        word_count[word] = word_count.get(word, 0) + 1 # setting the word count for each word in the list and adding it to the dict
    return word_count

spam_count = count_words(spam_words) # getting all the word counts for spam emails
ham_count = count_words(ham_words) # getting all the word counts for ham emails 

def word_probabilities(word_counts, total_words, smooth=1): # Need to find the probability of each word occurring in spam or ham
    word_probs = {}
    for word, count in word_counts.items(): # We need to loop through the items in word_count to figure out the probability of each word 
        word_probs[word] = (count + smooth) / (total_words + len(word_counts)) # Laplace smoothing of +1
    return word_probs 

total_spam_words = len(spam_words) # from previous spam list, how many spam words are there
total_ham_words = len(ham_words) # from previous ham list, how many ham words are there

spam_word_probs = word_probabilities(spam_count, total_spam_words) # probability of each word in previous_spam being a spam email 
ham_word_probs = word_probabilities(ham_count, total_ham_words) # probability of each word in previous_ham being a ham email 

pspam = len(previous_spam) / (len(previous_spam) + len(previous_ham)) # From previous emails, figuring out the probability of spam or ham
pham = len(previous_ham) / (len(previous_spam) + len(previous_ham)) # spam = 4/7, ham = 3/7 

def classifier(email, spam_word_probs, ham_word_probs, pspam, pham): # put it all together for new_emails
    email_words = email.split() # split up new emails 
    spam_prob = pspam # previously found spam probabilities
    ham_prob = pham # previously found ham probabilities
    
    for word in email_words: # loop through all the words in the email         
        if word in spam_word_probs: # If the word is in the spam_word dict, take that and multiply out the probabilitiy
            spam_prob *= spam_word_probs[word] # multiplying this by the probability stored in the dict
        else:
            spam_prob *= 1 / (total_spam_words + len(spam_count))  # smoothing for unknown words using laplace smoothing, for words not found in data
        if word in ham_word_probs: # same as above
            ham_prob *= ham_word_probs[word]
        else:
            ham_prob *= 1 / (total_ham_words + len(ham_count)) # same smoothing as above
    
    if spam_prob > ham_prob: # I think the best way to do this is just to determine if p(spam) > p(ham), then it's probably spam 
        return "Spam"
    else:
        return "Ham"

def save_new_info(classification, email): # updating the saved_ham, saved_spam lists with the classification that we found for the "learning"
    if classification == "Spam":
        saved_spam.append(email)
    else:
        saved_ham.append(email)

for email in new_emails['spam']:
    classification = classifier(email, spam_word_probs, ham_word_probs, pspam, pham) # for each new email, determine if Spam
    print(str({email}) + " is likely " + str({classification})) # Print the output
    save_new_info(classification, email) # save info in the new saved_spam or saved_ham list

for email in new_emails['ham']: # same as above
    classification = classifier(email, spam_word_probs, ham_word_probs, pspam, pham)
    print(str({email}) + " is likely " + str({classification}))
    save_new_info(classification, email)

# The learning aspect is tricky - I just saved the outputs in saved_spam and saved_ham, so the "previous_spam" and "previous_ham" if this were
# to be used more regularly and with different emails would be replaced with saved_spam and saved_ham. 

{'renew your password'} is likely {'Spam'}
{'renew your vows'} is likely {'Spam'}
{'benefits of our account'} is likely {'Ham'}
{'the importance of physical activity'} is likely {'Ham'}


In [7]:
print("The updated SPAM email list is now: " + str(saved_spam + previous_spam)) # printing the new saved_spam list with the emails we just determined
print("The updated HAM email list is now: " + str(saved_ham + previous_ham))

The updated SPAM email list is now: ['renew your password', 'renew your vows', 'send us your password', 'review our website', 'send your password', 'send us your account']
The updated HAM email list is now: ['benefits of our account', 'the importance of physical activity', 'Your activity report', 'benefits physical activity', 'the importance vows']


<hr style=\"border:2px solid gray\"> </hr>

## Reflection

Write you reflection in below cell.

This was an interesting assignment that took me a lot of time to work through. I followed the core resources guide on the Naive Bayes Classifier and found a great video about this on the StatQuest YouTube channel, which was very helpful. For words that were not found in the training data, I struggled with what to do - the core resources example used (1-prob), where the prob was the probability that the email appeared in spam/ham, respectively. Ultimately, I decided to go with the formula 1/total spam + number of unique words in spam to ensure that any words not in the training data wouldn't result in a zero probability (since we are multiplying all of these probabilities, having a zero would throw off the result as 0). 

Once I could break down exactly what to calculate, it was not too bad, but figuring out the smoothing components and putting it all together was tough.

I could not figure out a completely autonomous way to self-update the list, so I saved the new classified words in a new list instead. If the user wants to run new emails, one would just load the saved_spam and saved_ham as the previous emails to get this to work, but as mentioned, I was unable to automate this. 

I referenced a lot of my peer's code, mostly Henry's and Jakub, in coming up with a solution, and as mentioned, I used YouTube and the core resources to figure out what I was doing in the first place.